In [1]:
# import packages

import numpy as np
import random
import math
import matplotlib.pyplot as plt
import os
import neat
import pickle

In [2]:
## Define constants


# Runge Kutta variables
# initial time
t0 = 0
# final time
tf = 20
# time increments
h = 0.01
# maximum driving force available
max_F = 10

# Fitness coefficients
# alpha determines the cost of dropping the pole below 90 degrees
alpha = 1
# beta determines the cost of moving the cart away from the centre (quadratic penalty)
beta = 0.1

# force of gravity
g = 9.8

# Initial State Variables
# mass of cart, initial velocity, initial position
m0 = 1
x_ = 0
x0 = 0

# mass of pole 1 head, initial angular velocity, initial angular position, length
m1 = 0.1
theta1_range = [0.0785] # starting position range
d_theta1_max = 0 # max starting angular velocity (r/s)
#theta1_ = 0
#theta1 = 0.1
l1 = 1
# mass of pole 2 head, initial angular velocity, initial angular position, length
m2 = 0#0.01
d_theta2_max = 0
theta2_range = [0]
#theta2_ = 0
l2 = 0.1


# Simulation constraints
xMAX = 2.4 # max track extent
theta1MAX = 0.6283
theta2MAX = 0.6283 # approx 36 degrees



In [3]:
# forcing function
def F(force):
    
    # ensure force does not exceed abs max force
    force_limited = max(min(force, max_F), -max_F)

    return force_limited


In [4]:
# define fitness function to minimize or maximize
# in this case minimize the costs

#Impulse
def impulse_penalty(force):
    # where F is the force function
    imp = h * abs(force)
    #turn off forcing penalty
    #imp = 0
    
    return imp
    
def inverted_penalty(X):
    # check if either pole is inverted (tipped below 90 degrees)
    theta1 = X[1]
    theta2 = X[2]
    sigma = (math.cos(theta1) < 0) or (math.cos(theta2) < 0)
    
    # penalty equal to maximum force available
    ivp = alpha*h * max_F *sigma
    return ivp
    
def centrality_penalty(X):
    # penalty quadratic in distance from starting position
    cxp = beta*h * abs(X[0] - x0) ** 2
    return cxp

def angular_deflection_penalty(X):
    theta1 = X[1]
    theta2 = X[2]
    #adp = h*(abs(math.sin(theta1 / 2)) + abs(math.sin(theta2 / 2)))
    adp = h*abs(math.sin(theta1 / 2))
    return adp


def fitness_penalty(X,force, t):
    ##imp = impulse_penalty(force)
    ##ivp = inverted_penalty(X)
    cxp = centrality_penalty(X)
    adp = angular_deflection_penalty(X)
    
    ##fitness = imp + ivp + cxp
    fitness = cxp + adp

    return fitness

# when a trial has completed pass in all state variables as well as the final number of time increments j
def fitness_calc(X_all, j):
    
    fp1 = j / (tf/h)
    
    if j < 1000:
        fp2 = 0
    else:
        # take the sum of the absolute values of x, x_, theta1, theta1_
        # this is taken over the last 100 time steps of the simulation
        
        ## default value
        #fp2 = 0.75 / np.sum(np.sum(abs(X_all[-100:,:]), axis=0)[[0,1,3,4]])

        ## new longer value
        #fp2 = 7.5 / np.sum(np.sum(abs(X_all[-1000:,:]), axis=0)[[0,1,3,4]])
        
        ## new longer value, quadratic in distance
        fp2 = 7.5 / (np.sum(np.sum(abs(X_all[-1000:,:]), axis=0)[[1,3,4]]) + 5*np.sum(abs(X_all[-1000:,:])**2, axis=0)[0])
        
        
    return 0.1*fp1+0.9*fp2

In [5]:
tf/h

2000.0

The change of state vector is given by

dX/dt = \[f0(t,x0), f1(t,x1), f2(t,x2), f3(t,x3), f4(t,x4), f5(t,x5)]

Integreating this via fourth order Runge Kutta (RK4) will result in an updated state vector at all time steps.

In [6]:
# define functions f1 through f6

# The inputs are a timestep, t, and the state variables, X_n at some time n

# For f0, f1, f2 the functions are simly 
# equal to the variables (x3, x4, x5) = (x_, theta1_, theta2_)

def f0(t, X, x__):
    x_ = X[3]
    return x_
def f1(t, X, x__):
    theta1_ = X[4]
    return theta1_
def f2(t, X, x__):
    theta2_ = X[5]
    return theta2_


# For f3, f4, f5 we solve the Lagrangian equations of motion for x'', theta1'' and theta2''

# total mass
M = m0+m1+m2

def f3(t, X, force):
    theta1 = X[1]
    theta2 = X[2]
    theta1_ = X[4]
    theta2_ = X[5]
    
    cos_theta1 = math.cos(theta1)
    cos_theta2 = math.cos(theta2)
    sin_theta1 = math.sin(theta1)
    sin_theta2 = math.sin(theta2)
    
    denom = M - m1*cos_theta1**2 - m2*cos_theta2**2
    numer = force + m1*sin_theta1*(g*cos_theta1-l1*theta1_**2) + \
            m2*sin_theta2*(g*cos_theta2-l2*theta2_**2)
    x__ = numer / denom
    return x__

def f4(t, X, force):
    x__ = f3(t, X, force)
    theta1 = X[1]
    theta1__ = 1/l1 *(x__ * math.cos(theta1) + g*math.sin(theta1))
    return theta1__
    
def f5(t, X, force):
    x__ = f3(t, X, force)
    theta2 = X[2]
    theta2__ = 1/l2 *(x__ * math.cos(theta2) + g*math.sin(theta2))
    return theta2__

# shorthand notations

def f3_q(t, X, x__):
    return x__

def f4_q(t, X, x__):
    theta1 = X[1]
    theta1__ = 1/l1 *(x__ * math.cos(theta1) + g*math.sin(theta1))
    return theta1__

def f5_q(t, X, x__):
    theta2 = X[2]
    theta2__ = 1/l2 *(x__ * math.cos(theta2) + g*math.sin(theta2))
    return theta2__


In [7]:
# Create a forth order Runge-Kutta algorithm to update the state variables

def RK4(t, X_n, h, force):
    """Takes a state vector, X_n, the time, t, and a step size h, 
    and returns an updated state vector"""
    # RK form
    # X_{n+1} = X_n + (1/6)*(k1+2*k2+2*k3+k4)
    
    func_list = [f0,f1,f2,f3_q,f4_q,f5_q]
    # pass components of X_n, t into func_list
    
    x__ = f3(t, X_n, force)
    k1 = np.array([h*f(t, X_n, x__) for f in func_list])
    x__ = f3(t+h/2, X_n+k1/2, force)
    k2 = np.array([h*f(t+h/2, X_n+k1/2, x__) for f in func_list])
    x__ = f3(t+h/2, X_n+k2/2, force)
    k3 = np.array([h*f(t+h/2, X_n+k2/2, x__) for f in func_list])
    x__ = f3(t+h, X_n+k3, force)
    k4 = np.array([h*f(t+h, X_n+k3, x__) for f in func_list])

    X_new = X_n + 1/6 * (k1+2*k2+2*k3+k4)
    
    return X_new



In [8]:
def create_initial_state(theta1_range, theta2_range, d_theta1_max, d_theta2_max):
    """Randomly generates an initial state based on ranges of values """
    
    # initial angles between 0 and 360 degrees
    #theta1 = random.random()*2*math.pi
    #theta2 = random.random()*2*math.pi
    
    theta1 = random.random()*(max(theta1_range) - min(theta1_range)) + min(theta1_range)
    theta2 = random.random()*(max(theta2_range) - min(theta2_range)) + min(theta2_range)
    
    # initial angular velocities between 0 and d_theta_max
    theta1_ = random.random()*d_theta1_max
    theta2_ = random.random()*d_theta2_max
    
    # set cart position and initial velocity to zero
    #x0 = 0
    #x_ = 0
    
    X0 = np.array([x0, theta1, theta2, x_, theta1_, theta2_])
    
    return X0
    

In [9]:
#__MAIN LOOP__

# Call RK4 with X0 as the initial state vector. Continue to update the vector, appending results
# while t+n*h <= tf



def main_function(genomes, config):
    """ The function which calls the RK4 function and calculates state vectors for
    the system. At each timestep a neural net updates the system forcing term F() at 
    time i. The inputs to the net are the current position variables of the system, 
    as well as the two previous time steps. The output of the net is a forcing term 
    which is capped at F_max."""

    ###### NEED to generate a set of 100 random initial state vectors X0 to loop over for each 
    #      genome. Fitness is sum of all fitnesses
    

    X0 = create_initial_state(theta1_range, theta2_range, d_theta1_max, d_theta2_max)
        
        
    # run simulation on each genome
    for _, g in genomes:
        ##net = neat.nn.FeedForwardNetwork.create(g, config)
        net = neat.nn.RecurrentNetwork.create(g, config)
        g.fitness = 0
            
        # initial state vector with previous two time steps
        X_n = X0
        t = t0
        X_all = X0
            
        # number of iterations the trial is run for 
        j = 0
            
        while t  < tf:
            # increment time
            t += h
            j+= 1
                
            # get forcing term, F, from neural net
            # net is fed position state vector at current time and previous 2 steps
            # first (and only) output is force term
                
            force = net.activate(X_n[:3])[0]
            force_scaled = force * max_F
                
                
            # update state vector X_n
            X_n = RK4(t, X_n, h, force_scaled)
                
            # append new state to record of all states
            X_all = np.vstack((X_all,X_n))
 
                
            # check if thresholds exceeded, break loop if they are
            if abs(X_n[0]) > xMAX:
                break
            if abs(X_n[1]) > theta1MAX:
                break
            if abs(X_n[2]) > theta2MAX:
                break
                    
        # increment fitness amount by fitness of trial
        fitness = fitness_calc(X_all, j)*1000
        g.fitness = fitness


### NEAT Component

Some test starting values

Initial population of genomes = 500
Number of randomized states for each genome to trial = 10
Number of generations = 50

In [10]:
def run(config_path):
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, \
                               neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)
    # generate a population
    p = neat.Population(config)
    
    # create stats reporters
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    
    # run fitness function
    # function, number of generations
    winner = p.run(main_function, 500)
    
    # show winner config & stats
    print('\nBest Genome:\n{!s}'.format(winner))
    
    # pickle the winner
    pickle.dump(winner, open("winning_model_r.p", "wb"))
    
    # return winner
    return winner
    

if __name__ == "__main__":
    # get local directory
    local_dir = os.path.abspath('')
    config_path = os.path.join(local_dir, "config-recurrent.txt")
    winning_model = run(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: 0.88030 stdev: 0.55649
Best fitness: 4.70000 - size: (3, 14) - species 1 - id 5
Average adjusted fitness: 0.074
Mean genetic distance 2.874, standard deviation 0.438
Population of 503 members in 26 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    14      4.7    0.073     0
     2    0    28      2.9    0.044     0
     3    0    12      3.3    0.067     0
     4    0     6      0.9    0.006     0
     5    0    13      1.9    0.083     0
     6    0    34      4.1    0.076     0
     7    0    18      1.6    0.036     0
     8    0     9      2.1    0.032     0
     9    0    91      3.3    0.208     0
    10    0     1      0.6    0.000     0
    11    0     2      0.6    0.000     0
    12    0    10      2.1    0.052     0
    13    0    15      2.5    0.082     0
    14    0    46      1.8    0.137     0
    15    0     2      2.6    0.045     0
    16    

Population's average fitness: 1.70160 stdev: 1.24668
Best fitness: 7.45000 - size: (2, 6) - species 23 - id 2975
Average adjusted fitness: 0.126
Mean genetic distance 2.500, standard deviation 0.636
Population of 504 members in 26 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    6    11      4.8    0.121     0
     2    6    18      2.9    0.141     5
     3    6    15      6.3    0.109     4
     4    6     6      3.7    0.103     3
     5    6    45      5.8    0.179     0
     6    6    33      4.3    0.201     2
     7    6    49      4.9    0.195     0
     8    6    46      4.9    0.116     3
     9    6    35      6.2    0.307     0
    10    6     2      0.6    0.000     6
    11    6     2      0.6    0.000     6
    12    6    21      4.0    0.174     4
    13    6    19      3.5    0.139     1
    14    6    20      3.4    0.162     4
    15    6     2      0.6    0.000     6
    16    6     1      0.8    0.017     5
    

Population's average fitness: 2.35040 stdev: 2.06554
Best fitness: 9.50000 - size: (4, 11) - species 12 - id 4235

Species 3 with 18 members is stagnated: removing it
Average adjusted fitness: 0.178
Mean genetic distance 2.331, standard deviation 0.597
Population of 498 members in 18 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   12    14      6.1    0.121     2
     4   12     7      2.6    0.134     1
     5   12    17      4.3    0.124     6
     6   12    35      9.1    0.245     1
     7   12    44      1.6    0.013     2
     8   12    63      7.7    0.271     1
     9   12    31      8.7    0.363     0
    12   12    23      9.5    0.178     3
    13   12    17      3.5    0.126     7
    14   12     9      3.9    0.101     1
    17   12    36      9.2    0.286     2
    19   12    15      3.1    0.109     6
    20   12    28      8.7    0.191     3
    22   12    26      4.3    0.175     1
    23   12    77      8.7    0.20

Population's average fitness: 2.65932 stdev: 2.11877
Best fitness: 11.25000 - size: (3, 9) - species 17 - id 6356
Average adjusted fitness: 0.169
Mean genetic distance 2.202, standard deviation 0.563
Population of 499 members in 12 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   20    20      6.7    0.092     7
     4   20    23      6.7    0.213     2
     6   20    41      9.5    0.225     0
     7   20   113      9.8    0.277     0
     8   20    30      3.2    0.106     4
     9   20    50      8.7    0.232     8
    13   20    27      4.3    0.141     5
    14   20    18      4.5    0.062     9
    17   20    43     11.3    0.208     7
    19   20    26      4.3    0.109     7
    22   20    33      4.3    0.144     9
    23   20    75      9.1    0.216     3
Total extinctions: 0
Generation time: 2.438 sec (2.464 average)

 ****** Running generation 21 ****** 

Population's average fitness: 2.89509 stdev: 2.05013
Best fitness: 

Population's average fitness: 3.19440 stdev: 2.29057
Best fitness: 13.65000 - size: (4, 7) - species 7 - id 13900

Species 23 with 43 members is stagnated: removing it
Average adjusted fitness: 0.190
Mean genetic distance 2.270, standard deviation 0.627
Population of 500 members in 5 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     6   31    96      8.8    0.103     9
     7   31   149     13.7    0.210     2
    13   31    34      4.3    0.111     8
    17   31   103     12.9    0.242     2
    19   31   118     13.0    0.284     1
Total extinctions: 0
Generation time: 3.225 sec (2.758 average)

 ****** Running generation 32 ****** 

Population's average fitness: 3.68540 stdev: 2.46354
Best fitness: 13.65000 - size: (4, 7) - species 7 - id 13900

Species 6 with 96 members is stagnated: removing it
Average adjusted fitness: 0.242
Mean genetic distance 2.293, standard deviation 0.680
Population of 499 members in 4 species:
   ID   age  si

...

Population's average fitness: 17.86192 stdev: 30.90945
Best fitness: 123.49416 - size: (8, 12) - species 125 - id 182404

Species 158 with 17 members is stagnated: removing it
Average adjusted fitness: 0.135
Mean genetic distance 2.872, standard deviation 0.791
Population of 501 members in 7 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   100  193    53    116.5    0.085    83
   125  111    63    123.5    0.121    68
   151   32    99    110.1    0.240     3
   153   26   100    111.3    0.206     3
   154   22    56    108.2    0.088     6
   159   12    94    106.4    0.137     0
   160    4    36    106.6    0.070     2
Total extinctions: 0
Generation time: 16.617 sec (17.458 average)

 ****** Running generation 447 ****** 

Population's average fitness: 18.55936 stdev: 31.60811
Best fitness: 123.49416 - size: (8, 12) - species 125 - id 182404
Average adjusted fitness: 0.135
Mean genetic distance 2.890, standard deviation 0.775
Popula

Population's average fitness: 16.60619 stdev: 29.93207
Best fitness: 123.49416 - size: (8, 12) - species 125 - id 182404
Average adjusted fitness: 0.103
Mean genetic distance 2.936, standard deviation 0.713
Population of 501 members in 6 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   100  205    87    116.5    0.099    95
   125  123   114    123.5    0.140    80
   159   24   129    108.9    0.160     1
   160   16   103    111.0    0.156     6
   161    3    35      5.1    0.020     0
   162    2    33     34.5    0.042     0
Total extinctions: 0
Generation time: 15.536 sec (17.968 average)

 ****** Running generation 459 ****** 

Population's average fitness: 14.55359 stdev: 27.23473
Best fitness: 123.49416 - size: (8, 12) - species 125 - id 182404
Average adjusted fitness: 0.099
Mean genetic distance 2.943, standard deviation 0.728
Population of 501 members in 6 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  ==

Population's average fitness: 16.34293 stdev: 30.27372
Best fitness: 123.49416 - size: (8, 12) - species 125 - id 182404
Average adjusted fitness: 0.121
Mean genetic distance 2.911, standard deviation 0.774
Population of 500 members in 7 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   100  217    66    116.5    0.102   107
   125  135    66    123.5    0.082    92
   161   15    36     12.0    0.037     9
   162   14   125    105.8    0.106     6
   163    8    41    113.4    0.166     3
   164    7    98    111.0    0.223     3
   165    5    68    110.8    0.133     3
Total extinctions: 0
Generation time: 15.506 sec (15.699 average)

 ****** Running generation 471 ****** 

Population's average fitness: 18.08125 stdev: 31.94672
Best fitness: 123.49416 - size: (8, 12) - species 125 - id 182404

Species 161 with 36 members is stagnated: removing it
Average adjusted fitness: 0.149
Mean genetic distance 2.855, standard deviation 0.799
Popula

Population's average fitness: 17.30832 stdev: 31.32703
Best fitness: 123.49416 - size: (8, 12) - species 125 - id 182404
Average adjusted fitness: 0.097
Mean genetic distance 2.823, standard deviation 0.643
Population of 501 members in 8 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   125  145    62    123.5    0.081   102
   166    9    66    111.5    0.102     7
   167    9    91    106.9    0.150     1
   168    8    34    102.8    0.054     1
   169    7    86    120.8    0.248     2
   170    7    99    102.3    0.050     0
   171    6    48    101.3    0.074     0
   172    2    15      6.4    0.021     0
Total extinctions: 0
Generation time: 16.902 sec (18.706 average)

 ****** Running generation 481 ****** 

Population's average fitness: 19.44135 stdev: 34.03831
Best fitness: 123.49416 - size: (8, 12) - species 125 - id 182404
Average adjusted fitness: 0.130
Mean genetic distance 2.728, standard deviation 0.605
Population of 499 m

Population's average fitness: 18.77094 stdev: 32.93382
Best fitness: 123.49416 - size: (8, 12) - species 125 - id 182404
Average adjusted fitness: 0.128
Mean genetic distance 2.719, standard deviation 0.645
Population of 499 members in 8 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   125  156    52    123.5    0.114   113
   167   20   101    115.2    0.235     0
   168   19    58    105.7    0.088     6
   170   18   104    120.8    0.235     5
   171   17    55    107.3    0.097     2
   172   13    60    104.2    0.114     8
   173    7    43    111.7    0.115     2
   174    6    26     10.2    0.028     0
Total extinctions: 0
Generation time: 17.939 sec (18.821 average)

 ****** Running generation 492 ****** 

Population's average fitness: 18.42000 stdev: 31.27804
Best fitness: 123.49416 - size: (8, 12) - species 125 - id 182404
Average adjusted fitness: 0.131
Mean genetic distance 2.684, standard deviation 0.627
Population of 500 m